### Overview
* <a href="#section1">What is a grammar, how does ambiguity emerge?</a>
* <a href="#section2">What is dependency grammar?</a>
    * contrast with constituency/CFG
    * mention the assumptions of DG, any issues there
    * briefly mention the shift-reduce parsing method SpaCy uses
* <a href="#section3">why is it useful?</a>
    * deal with languages with free word order
        * phrase structure rules/constituency parsers explode with rules.
        * eg NP -> ADJ NN and NN ADJ
    * applications: 
        * chunking for information retrieval: 
            * which words modify the noun or verb?
            * extract noun phrases, verb phrases, adjective phrases, and prepositional phrases
            * see <a href="#relationextractor">example</a>
        * question answering 
        * semantic parsing        
        * feature engineering for coreference resolution:
            * When Bob goes to the movies [he] likes to buy popcorn with extra butter.          
* <a href="#section4"> Dependency Grammar with SpaCy:</a>
    * Traversing paths
    * Grabbing entities
    * Exercise:
        * extract all noun phrases from a sentence

In [11]:
from warnings import warn
from IPython.display import HTML
def from_scratch():
    !pip install gensim >> ~/gensim.log
    !pip install spacy >> ~/spacy.log
    !python -m spacy download en >> ~/spacy.log
    !jupyter nbextension enable --py --sys-prefix widgetsnbextension
    warn("Refresh notebook to reflect widget availability.")

    
from_scratch()

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


/opt/conda/envs/python3/lib/python3.5/site-packages/ipykernel_launcher.py:8: UserWarning: Refresh notebook to reflect widget availability.
  


In [2]:
from static_html import syntax
HTML(syntax)

In [3]:
from static_html import morphology
HTML(morphology)

,Effect,Morpheme
,,
un,Negation,Prefix
reach,Verb,Root
able,To Adjective,Suffix


In [4]:
from static_html import morph_and_syntax
HTML(morph_and_syntax)

# Models of Grammar

In [5]:
from static_html import valency
HTML(valency)

### Context Free Grammar, production rules

|Root| |Components |
|-----||---------------------------------------|
| S   |$\rightarrow$|NP VP                                 |
| NP |$\rightarrow$| Pron or PropN or Det Nom   |
| VP  |$\rightarrow$|V or V NP or V NP NN or V PP |
| PP  |$\rightarrow$|Prep NP                        |
| etc |$\rightarrow$|etc                                   |

### Constituency and Dependency Grammars

In [123]:
from IPython.display import Image
Image(url='https://upload.wikimedia.org/wikipedia/commons/8/8e/Thistreeisillustratingtherelation%28PSG%29.png')

In [121]:
from static_html import dep_axioms
HTML(dep_axioms)

<a name="section3"</a>
### Applications of DG

<a name="section4"</a>
### DG with SpaCy

In [2]:
from IPython.display import HTML
from static_html import accessing_dependents
HTML(accessing_dependents)

In [79]:
from IPython.display import HTML
import urllib
from IPython.display import IFrame
html = """
<style>
#wrapper {{ width: 710px; height: 1420px; padding: 0; overflow: hidden; }}
#scaled-frame {{ width: 1000px; height: 2000px; border: 0px; }}
#scaled-frame {{
    zoom: 0.71;
    -moz-transform: scale(0.71);
    -moz-transform-origin: 0 0;
    -o-transform: scale(0.71);
    -o-transform-origin: 0 0;
    -webkit-transform: scale(0.71);
    -webkit-transform-origin: 0 0;
}}

@media screen and (-webkit-min-device-pixel-ratio:0) {{
 #scaled-frame  {{ zoom: 1;  }}
}}
</style>

<div id="wrapper" style="overflow: hidden; position: relative; left: -200px; width:1500px ;height: 300px"><iframe id="scaled-frame" src="{}/"></iframe></div>
"""

def displacy(text):
    body = {"model":'en', 'cpu':1, 'cph':1}
    query_string = urllib.parse.urlencode(body)
    url = "https://demos.explosion.ai/displacy/?{0}&text={1}"
    url = url.format(query_string, text.replace(" ", "%20"), )
    code = html.format(url)

    return HTML(code)

In [78]:
displacy("hey how are you")

In [81]:
import spacy
nlp = spacy.load('en')


## Appendix
<a name="relationextractor"></a>
### Relation Extractor

In [112]:
def has_subject_and_attribute(verb):
    children_depencies =list(map(lambda i: i.dep_, verb.children))
    return 'nsubj' in children_depencies and any(i in children_depencies for i in ('attr','acomp','dobj'))
def is_verb(token):
    return token.pos_ == 'VERB'
def extract_simple_relations(sentence):
    relations = []
    for VERB in filter(is_verb,sentence):
        if has_subject_and_attribute(VERB):
            #property_attribute = []
            for child in VERB.children:
                if child.dep_ == 'nsubj':
                    SUBJECT = child
                if child.dep_ in ('attr','acomp','dobj','pobj','prep'):
                    PROPERTY = list(child.subtree)
                    
            relation = {'subject':SUBJECT,'property':PROPERTY,'relation':VERB}
            relations.append(relation)
    return relations

doc = nlp('The man eats cake, but the woman eats ice cream.')
extract_relations(doc)

[{'property': [cake], 'relation': eats, 'subject': man},
 {'property': [ice, cream], 'relation': eats, 'subject': woman}]

In [113]:
extract_simple_relations(nlp('He bought a camera, but could not figure out how to use it'))

[{'property': [a, camera], 'relation': bought, 'subject': He}]

In [116]:
text = """
This coffee is very smooth and flavorful. 
I will be buying more as gifts and more for myself. 
I like this company and will be seeing what other flavors I can check out. 
So glad I took a chance on this coffee.
"""
extract_simple_relations(nlp(text))

[{'property': [very, smooth, and, flavorful],
  'relation': is,
  'subject': coffee},
 {'property': [more, as, gifts, and, more, for, myself],
  'relation': buying,
  'subject': I},
 {'property': [this, company], 'relation': like, 'subject': I},
 {'property': [what, other, flavors], 'relation': check, 'subject': I},
 {'property': [a, chance, on, this, coffee], 'relation': took, 'subject': I}]